In [1]:
import os
import torch
import trainers
import csv
!nvidia-smi

Tue Nov  2 20:21:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     On   | 00000000:3B:00.0 Off |                    0 |
| N/A   29C    P8    15W / 250W |    101MiB / 22698MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 6000     On   | 00000000:5E:00.0 Off |                    0 |
| N/A   

In [2]:
cuda_device_idx = "3"
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device_idx
!echo $CUDA_VISIBLE_DEVICES

3


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
save_loc = r'/scratch/jferrari/diffusion-ai-results/log-transform/diff-N-training'
if not os.path.isdir(save_loc):
    os.mkdir(save_loc)

load_loc = r"/scratch/jferrari/ten_twenty_data_set"
dataset_name = '1SourcesRdm'

if not os.path.isdir(save_loc):
    os.mkdir(save_loc)
save_loc = os.path.join(save_loc,dataset_name)
BATCH_SIZE = 25
NUM_WORKERS = 4
lr = 0.0001

In [4]:
trainer = trainers.DeepDiffusionTrainer(device=device, std_tr=0.01, s=512)

In [5]:
error_list, test_error, test_error_plus, test_error_minus, save_loc = \
        trainer.train_diff_solver(load_loc, save_loc, lr, BATCH_SIZE, NUM_WORKERS,
                                  dataset_name=dataset_name, transformation='log')

Save location: /scratch/jferrari/diffusion-ai-results/log-transform/diff-N-training/1SourcesRdm/2021-11-02-20-21-17


/scratch/jferrari/miniconda3_install/lib/python3.9/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


TypeError: my_loss() missing 1 required positional argument: 'target'

In [ ]:
with open(os.path.join(save_loc, 'train_error.csv'), 'w+') as f:
    writer = csv.writer(f)
    writer.writerow(error_list)
with open(os.path.join(save_loc, 'test_error.csv'), 'w+') as f:
    writer = csv.writer(f)
    writer.writerow(test_error)
if len(test_error_plus):
    with open(os.path.join(save_loc, 'test_N+1_error.csv'), 'w+') as f:
        writer = csv.writer(f)
        writer.writerow(test_error_plus)
if len(test_error_minus):
    with open(os.path.join(save_loc, 'test_N-1_error.csv'), 'w+') as f:
        writer = csv.writer(f)
        writer.writerow(test_error_minus)